## Interplanetary Logistics: Earth to Mars

In [1]:
from astropy import time

In [5]:
from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver

In [22]:
def total_delta_v(launch, arrival):
    """Calculate the total delta v for specific combination of launch date and arrival date
    """
    
    date_launch = time.Time(launch, scale="utc")
    date_arrival = time.Time(arrival, scale="utc")

    ss_earth = Orbit.from_body_ephem(Earth, date_launch)
    ss_mars = Orbit.from_body_ephem(Mars, date_arrival)

    man_lambert = Maneuver.lambert(ss_earth, ss_mars)

    return {
        'delta_v': man_lambert.get_total_cost().value, 
        'total_seconds': man_lambert.get_total_time().value
    }


In [26]:
class PricingEngine():
    def __init__(self, params):
        """Initialize pricing engine
        
        params specifies the parameters to be used to determine the price of the shipment.
        
        1. delta_v is in unit of km/s
        2. euros_per_delta_v is the cost for each km/s
        """
        self.delta_v = params.get('delta_v')
        self.euros_per_delta_v = params.get('euros_per_delta_v') or 1/50
        
    def _per_delta_v(self):
        
        return self.delta_v * self.euros_per_delta_v
    
    def price(self, method=None):
        """Calculate the price
        """
        if method is None:
            method = 'per_delta_v'
        
        methods = {
            'per_delta_v': self._per_delta_v
        }
        
        use_method = methods.get(method)
        if not use_method:
            raise Exception('No method defined for : {}'.format(use_method))
        else:
            euros = use_method()
            return euros
        
        

In [29]:
parameters = {
    'delta_v': total_delta_v('2019-12-08', '2022-01-01').get('delta_v')
}
pe = PricingEngine(parameters)

In [30]:
pe.price()

24940.31092366127